# Create Heatmaps for significant TCR Signaling genes

Pancancer heatmaps are created with circle size showing significance and color showing differences in median.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import gseapy as gp
import re
import sys 

import cptac
import cptac.utils as u

import plot_utils as p

# Step 1: Run GSEA for significant genes in at least 1 cancer

First read in sig_pval_all_proteins.csv into a df. This csv file contains only genes with a significant p-value in at least one cancer. Then run GSEA using a list of genes from the df.

In [2]:
root = R'~\Github\WhenMutationsDontMatter\PTEN\Step_3_trans_effect\csv'
sig_df = pd.read_csv(root+R'\sig_pval_heatmap.csv')

prot_list = list(sig_df.Proteomics) # list of genes with a sig pval in >= 1 cancer
prot_enr = gp.enrichr(gene_list = prot_list, description='Tumor_partition', gene_sets='NCI-Nature_2016', 
                       outdir='/Enrichr')

In [3]:
prot_enr.res2d.iloc[[12]]

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
12,TCR signaling in naive CD4+ T cells Homo sapie...,18/64,0.001059,0.017023,0,0,2.154347,14.758576,CD86;MAP4K1;CHUK;PDPK1;PRKCB;PTEN;WAS;CD3D;VAV...,NCI-Nature_2016


# Step 2: Get the list of significant genes 

In [4]:
i = 12
trans = prot_enr.res2d.Genes[i]
genes = trans.split(';')
print(prot_enr.res2d.Term[i])
print('total genes:',len(genes))

TCR signaling in naive CD4+ T cells Homo sapiens 0c2862fa-6196-11e5-8ac5-06603eb7f303
total genes: 18


# Step 3: Create HeatMap

Slice out genes from the DNA Replication pathway from the df with genes sig in > 1 cancer.

In [5]:
# sig > 1 cancer
bool_df = sig_df.Proteomics.isin(genes)
plot_df = sig_df[bool_df]
len(plot_df.Proteomics.unique())

18

In [6]:
# Only include p-values < a 
a = 0.01
plot_df = plot_df.loc[plot_df['P_Value'] <= a]

In [7]:
p.plotCircleHeatMap(plot_df, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=400, plot_width= 800)

Loading BokehJS ...

# Step 4: Create a HeatMap with both +/- differences in median

 Read in the pos_neg_df.csv to create a df with only genes that have a pos and neg difference in median in different cancers. Slice out the genes that have a pos and neg difference in median in the pathway using the list of genes with a significant p-value in the pathway. 

In [8]:
pos_neg_df = pd.read_csv(root+R'\pos_neg_df.csv')

In [9]:
get = pos_neg_df.Proteomics.isin(genes) # bool df where True has both pos and neg
genes_pn = pos_neg_df[get] # Keep only genes with pos and neg
genes_pn.Proteomics.unique()

array(['MAP4K1'], dtype=object)

In [10]:
# Only include p-values < a
genes_pn = genes_pn.loc[genes_pn['P_Value'] <= a]

In [11]:
p.plotCircleHeatMap(genes_pn, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=400, plot_width=400)

Loading BokehJS ...

# Sig in multiple cancers

In [12]:
mult_sig_df = pd.read_csv(root+R'\mult_sig_pval_heatmap.csv')

mult_sig_list = list(mult_sig_df.Proteomics.unique()) # list of genes with a sig pval in > 1 cancer
enr2 = gp.enrichr(gene_list = mult_sig_list, description='Tumor_partition', gene_sets='NCI-Nature_2016', 
                       outdir='/Enrichr')

In [13]:
enr2.res2d.iloc[[2]]

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
2,TCR signaling in naive CD4+ T cells Homo sapie...,8/64,0.000006,0.000452,0,0,8.038585,96.020522,MAP4K1;ZAP70;CD4;PTPRC;CHUK;PRKCB;PTEN;WAS,NCI-Nature_2016


In [26]:
i = 2
trans2 = enr2.res2d.Genes[i]
genes_mult = trans2.split(';')
print(enr2.res2d.Term[i])
print('total genes:',len(genes_mult))

TCR signaling in naive CD4+ T cells Homo sapiens 0c2862fa-6196-11e5-8ac5-06603eb7f303
total genes: 8


In [27]:
bool_df = mult_sig_df.Proteomics.isin(genes_mult)
plot_df2 = mult_sig_df[bool_df]
plot_df2.Proteomics.unique()

array(['PTEN', 'CHUK', 'PRKCB', 'WAS', 'PTPRC', 'CD4', 'ZAP70', 'MAP4K1'],
      dtype=object)

In [28]:
# Only include p-values < a
plot_df2 = plot_df2.loc[plot_df2['P_Value'] <= a]

In [30]:
p.plotCircleHeatMap(plot_df2, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=400, plot_width=500)

Loading BokehJS ...

In [36]:
imm = ["CD79A","CD79B","CD274","BLK","BTK","LCK","LYN","SYK","BLNK","CD19","PLCG2","PI3K","CD5","CD22","PCK","HS1","CCL3","CCL4","PTPN6","ZAP70","LCK","ITK","CD3E","CD3G","CD4","VAV1","GRB2","ADAP","GADS","LAT","SLP76","NFAT","PLCG1","PTPN22","SOS1","HLA-DP","HLA-DM","HLA-DOA","HLA-DOB","HLA-DQ","HLA-DRA","HLA-DRB1","GZMA","CD8A","PRF1","CIITA","INFGR1","JAK1","JAK2","STAT1","B2M","HLA-A","HLA-B","HLA-C","TAP1","TAP2","PSMB8","PSMB9","PSMB10","IFDR1"]

In [48]:
for e in imm:
    if e == 'MAP4K1':
        print('True')

In [ ]:
['CHUK', 'PRKCB', 'WAS', 'PTPRC', 'MAP4K1']

In [31]:
get = pos_neg_df.Proteomics.isin(genes_mult) # bool df where True has both pos and neg
genes_pn_mult = pos_neg_df[get] # Keep only genes with pos and neg
genes_pn_mult.Proteomics.unique()

array(['MAP4K1'], dtype=object)

In [32]:
# Only include p-values < a
genes_pn_mult = genes_pn_mult.loc[genes_pn_mult['P_Value'] <= a]

In [33]:
p.plotCircleHeatMap(genes_pn_mult, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=400, plot_width=400)

Loading BokehJS ...

# Check other pathways the genes (+/- and sig in mult. cancers) are part of

In [21]:
enr3 = gp.enrichr(gene_list = genes_mult, description='Tumor_partition', gene_sets='NCI-Nature_2016', 
                       outdir='/Enrichr')

In [22]:
enr3.res2d.head()

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
0,Signaling events mediated by TCPTP Homo sapien...,15/42,3.941340e-42,8.237400e-40,0,0,476.190476,45398.596743,CSF1R;STAT1;EIF2AK2;PIK3CD;PIK3R1;EGFR;VEGFA;P...,NCI-Nature_2016
1,SHP2 signaling Homo sapiens 85755aa4-6195-11e5...,8/57,1.650498e-17,1.724771e-15,0,0,187.134503,7231.414128,PIK3CA;STAT1;GRB2;PIK3R1;JAK3;EGFR;VEGFA;JAK1,NCI-Nature_2016
2,EGF receptor (ErbB1) signaling pathway Homo sa...,6/34,7.493580e-14,5.220527e-12,0,0,235.294118,7111.092855,PIK3CA;STAT1;PIK3CD;GRB2;PIK3R1;EGFR,NCI-Nature_2016
3,IL2-mediated signaling events Homo sapiens a2a...,6/55,1.602335e-12,8.372202e-11,0,0,145.454545,3950.481299,PIK3CA;STAT1;GRB2;PIK3R1;JAK3;JAK1,NCI-Nature_2016
4,IL4-mediated signaling events Homo sapiens cff...,6/60,2.761855e-12,1.154456e-10,0,0,133.333333,3548.682453,PIK3CA;STAT6;GRB2;PIK3R1;JAK3;JAK1,NCI-Nature_2016


In [23]:
i = 4
print(enr3.res2d.Term[i])
enr3.res2d.Genes[i]

IL4-mediated signaling events Homo sapiens cff33f50-6193-11e5-8ac5-06603eb7f303


'PIK3CA;STAT6;GRB2;PIK3R1;JAK3;JAK1'

In [24]:
# Look at p-values 
certain_gene = 'SYK'
all_df = pd.read_csv(root+R'\all_heatmap.csv')
gene_df = all_df.loc[all_df['Proteomics'] == certain_gene]
gene_df

,Proteomics,P_Value,Medians,Cancer
499,SYK,0.000320,-0.697550,Gbm
11499,SYK,0.001919,-0.225323,Hnscc
23264,SYK,0.021461,-0.489000,Luad
40690,SYK,0.675254,0.222300,Lscc
53912,SYK,0.905056,0.020400,Brca
54758,SYK,0.003901,0.405618,Ov
67986,SYK,0.085643,0.079000,En
82040,SYK,0.775873,0.034700,Colon
